torch 버전 : 2.0.0+cpu   
torchvision 버전 : 0.14.0+cpu  

In [4]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.0.0+cpu
0.14.0+cpu


In [5]:
import requests
import matplotlib.pyplot as plt
import time
import torch.nn.functional as F

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

In [6]:
# 0 : 물소   # 1 : 코끼리
# 2 : 코뿔소 # 3 : 얼룩말
CLASSES = ['0','1','2','3']

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

transform= T.Compose([
    T.Resize(800), # * 이유.
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # dataset을 확인하면 된다
])

# output bounding box 후처리
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [7]:
class DETRdemo(nn.Module):
    def __init__(self, num_classes, hidden_dim=256, nheads=8,
                num_encoder_layers=6, num_decoder_layers=6):
        super().__init__()
        self.backbone = resnet50()
        del self.backbone.fc 
        self.conv = nn.Conv2d(2048, hidden_dim, 1)
        self.transformer = nn.Transformer(hidden_dim, nheads, 
                                        num_encoder_layers, num_decoder_layers)
        self.linear_class = nn.Linear(hidden_dim, 92)
        self.final_linear_class = nn.Linear(92, num_classes + 1)
        
        self.linear_bbox = nn.Linear(hidden_dim, 4)
        self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim//2))
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim//2))
        
    def forward(self, inputs):
        x = self.backbone.conv1(inputs)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        
        x = self.backbone.layer1(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        h = self.conv(x)
        H, W = h.shape[-2:] 
        pos=torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
        ], dim=-1).flatten(0, 1).unsqueeze(1)
        h = self.transformer(pos+0.1*h.flatten(2).permute(2, 0, 1),
                            self.query_pos.unsqueeze(1)).transpose(0,1)
        
        return {'pred_logits': self.final_linear_class(self.linear_class(h)),
                'pred_boxes': self.linear_bbox(h).sigmoid()}

In [8]:
import pandas as pd

train = pd.read_csv('./input/dataset/new_train.csv').drop(columns=["index"])
test = pd.read_csv('./input/dataset/new_test.csv').drop(columns=["index"])
train.head()

,filename,label,center_x,center_y,width,height
0,1,1,0.438281,0.529551,0.814063,0.704492
1,2,3,0.541016,0.524324,0.849219,0.672973
2,3,2,0.492500,0.517045,0.695000,0.725000
3,4,2,0.585156,0.525000,0.692187,0.785135
4,5,3,0.385547,0.645270,0.614844,0.698649


In [9]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스
import numpy as np

class ImageDataset(Dataset):
    # 초기화 메서드 (생성자)
    def __init__(self, df, img_dir ='./', transform=None):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터셋 크기 반환 메서드    
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0].zfill(4) # 이미지 ID
        img_path = self.img_dir + img_id + '.jpg' # 이미지 파일 경로
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        
        label = self.df.iloc[idx, 0] # 라벨값
        center_x = self.df.iloc[idx, 1] # 중심 x
        center_y = self.df.iloc[idx, 2] # 중심 y
        width = self.df.iloc[idx, 3] # 너비
        height = self.df.iloc[idx, 4] # 높이 
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        return image, center_x, center_y, width, height, label

In [10]:
dataset_train = ImageDataset(train, img_dir='./input/dataset/train/', transform=transform)
dataset_valid = ImageDataset(test, img_dir='./input/dataset/test/', transform=transform)

In [11]:
import random

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [12]:
from torch.utils.data import DataLoader

batch_size = 1

loader_train = DataLoader(dataset_train, batch_size=batch_size,
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=1)

loader_test = DataLoader(dataset_valid, batch_size=batch_size,
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [14]:
model = DETRdemo(num_classes=4)
state_dict = torch.hub.load_state_dict_from_url(
    url='https://dl.fbaipublicfiles.com/detr/detr_demo-da2a99e9.pth',
    map_location='cpu', check_hash=True)

# 추출한 파라미터를 모델에 로드합니다
model.load_state_dict(state_dict, strict=False)
model.to(device)

DETRdemo(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [15]:
class MultiBoxLoss(nn.Module):
    def __init__(self):
        super(MultiBoxLoss, self).__init__()

    def forward(self, predicted_boxes, predicted_labels, target_boxes, target_labels):
        # predicted_boxes: 예측된 bounding box
        # predicted_labels: 예측된 클래스 레이블
        # target_boxes: 실제 bounding box
        # target_labels: 실제 클래스 레이블

        # Smooth L1 Loss 계산
        smooth_l1_loss = F.smooth_l1_loss(predicted_boxes, target_boxes)

        # Cross Entropy Loss 계산
        cross_entropy_loss = F.cross_entropy(predicted_labels, target_labels)

        # MultiBox Loss 계산 (Smooth L1 Loss + Cross Entropy Loss)
        multibox_loss = smooth_l1_loss + cross_entropy_loss

        return multibox_loss

In [16]:
# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0006, weight_decay=0.0001)

In [17]:
from transformers import get_cosine_schedule_with_warmup
epochs = 5

# 스케줄러 생성
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                   num_warmup_steps=len(loader_train)*3,
                                   num_training_steps=len(loader_train)*epochs)

아직 코드 다 안짰는데 여기 부분 수정해야함

In [18]:
#detect
def detect(im, model, transform):
    
    # input image를 정규화해줍니다. (batch-size : 1)
    img = transform(im).unsqueeze(0)
    
    # demo의 경우 aspect ratio를 0.5와 2사이만 지원합니다.
    # 이 범위 밖의 이미지를 사용하고 싶다면 maximum size을 1333이하로 rescaling해야 합니다.
    assert img.shape[-2] <= 1600 and img.shape[-1] <= 1600
    
    # model을 통과시킵니다. 
    outputs = model(img)
    
    # 70 % 이상의 정확도를 가진 예측만 남깁니다.
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.7
    
    # 0과 1사이의 boxes 값을 image scale로 확대합니다.
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
    return probas[keep], bboxes_scaled

#visualize
def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [19]:
for tmp in loader_train:
    print(tmp)
    break

In [18]:
from tqdm.notebook import tqdm # 진행률 표시 막대

# 총 에폭만큼 반복
for epoch in range(epochs):
    # == [ 훈련 ] =================================
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0  # 에폭별 손실값 초기화 (훈련 데이터용)
    
    # '반복 횟수' 만큼 반복
    for tmp in tqdm(loader_train): 
        print(tmp)
        break
    #     # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당
    #     images = images.to(device)
    #     box_regions = box_regions.to(device)
    #     labels = labels.to(device)
        
    #     # 옵티마이저 내 기울기 초기화
    #     optimizer.zero_grad()
    #     # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    #     outputs = model(images)
    #     # 손실 함수를 활용해 outputs 와 labels 의 손실값 계산
    #     loss_fn = MultiBoxLoss()
        
    #     loss = loss_fn(predicted_boxes, predicted_labels, target_boxes, target_labels)
    #     # 현재 배치에서의 손실 추가 (훈련 데이터용 )
    #     epoch_train_loss += loss.item()
    #     loss.backward() # 역전파 수행
    #     optimizer.step() # 가중치 갱신
    # # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    # # == [ 검증 ] ====
    # model.eval() # 모델을 평가 상태로 설정
    # epoch_valid_loss = 0 # 에폭별 손실값 초기화(검증 데이터용)
    # preds_list = [] # 예측 확률값 저장용 리스트 초기화 
    # true_onehot_list = []
    
    # with torch.no_grad(): # 기울기 계산 비활성화
    #     # 미니배치 단위로 검증
    #     for images, labels in loader_valid:
    #         images = images.to(device)
    #         labels = labels.to(device)
            
    #         outputs = model(images)
    #         loss = criterion(outputs, labels)
    #         epoch_valid_loss += loss.item()
            
    #         preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
    #         # 실젯값 (원-핫 인코딩 형식)
    #         true_onehot = torch.eye(4)[labels.to('cpu')].numpy()
    #         # 예측 확률값과 실젯값 저장
    #         preds_list.extend(preds)
    #         true_onehot_list.extend(true_onehot)
    #     # 검증 데이터 손실값 및 ROC AUC 점수 출력
    #     print(f'''에폭 [{epoch+1}/{epochs}]
    #           검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} 
    #           검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}''')

  0%|          | 0/2127 [00:00<?, ?it/s]